# DEG analysis using DESeq2


## Suggested reading
- Deseq2 tutorial video part 1: https://www.youtube.com/watch?v=UFB993xufUU&t=38s
- Deseq2 tutorial video part 2: https://www.youtube.com/watch?v=Gi0JdrxRq5s
- [optional] DESeq2 documentation: http://bioconductor.org/packages/devel/bioc/vignettes/DESeq2/inst/doc/DESeq2.html

## Things I do here
1. I load R packages using rpy2, make sure you installed those packages in R
2. I load gene_count_df and make a sample_df, both needed for DESeq2
3. I selected a pair of sample to run DESeq2, and the result is saved into a CSV file

## Note
- This notebook is also a demo of how we can use python and R in the same notebook
- This notebook can be run on other sample pairs, all you need to change is the parameter cell below. Try to do this with papermill automatically (either use command line interface of papermill or use papermill's `execute_notebook` function in a separate notebook)
- I usually call this parameterized notebook "template" and use it in another "master" notebook with the `execute_notebook` functions

In [1]:
import pandas as pd

import rpy2.robjects as ro
from rpy2.robjects.packages import importr

# this line is important, its called ipython magic, 
# and rpy2.ipython has spetial magic "%%R" that allows me to execute R directly later in this python notebook!
%load_ext rpy2.ipython

In [2]:
!which R

/usr/local/bin/R


**You need to install DESeq2 in R. Make sure you are using same R version as the one printed above.**

```R
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")

BiocManager::install("DESeq2")
BiocManager::install("apeglm")
BiocManager::install("IHW")
```

In [3]:
# import R packages using ryp2
importr('DESeq2')
importr('apeglm')
importr('IHW')

rpy2.robjects.packages.Package as a <module 'IHW'>

## Parameter

This is the key parameter cell of this notebook, you can parameterize this notebook with papermill

In [4]:
# here is the default value, will be replaced if this notebook is executed by papermill. Why?
sample_pair = ['E10.5', 'E14.5']

## Load data and prepare

In [5]:
gene_count_df = pd.read_csv('./gene_raw_count_table.csv.gz', index_col=0)
gene_count_df.head()

,forebrain_P0_2,forebrain_E10.5_1,forebrain_E14.5_2,forebrain_P0_1,forebrain_E14.5_1,forebrain_E10.5_2
gene_id,,,,,,
ENSMUSG00000000001.4,8312,16156,9459,8661,10751,14368
ENSMUSG00000000003.15,0,0,0,0,0,0
ENSMUSG00000000028.15,284,3997,1038,230,1428,3666
ENSMUSG00000000031.16,611,4061,909,578,1059,4205
ENSMUSG00000000037.17,142,355,225,181,289,283


### Basic filter to remove low exp gene

If a gene expression is too low, it is not meaningful to report that. DESeq2 will filter genes automatically, but prefiltering will make it run faster, as suggested by the DESeq2 doc.

In [6]:
sample_mean_cutoff = 1

print('Before', gene_count_df.shape[0])

gene_count_df = gene_count_df[gene_count_df.mean(axis=1) > sample_mean_cutoff].copy()

print('After', gene_count_df.shape[0])

Before 54331
After 26782


### make sample metadata based on gene_count_df.columns

In [7]:
sample_meta = {}
for sample in gene_count_df.columns:
    _, time, rep = sample.split('_')
    sample_meta[sample] = {'devTime': time, 'rep': rep}
sample_df = pd.DataFrame(sample_meta).T.reindex(gene_count_df.columns) # make sure order

sample_df

,devTime,rep
forebrain_P0_2,P0,2
forebrain_E10.5_1,E10.5,1
forebrain_E14.5_2,E14.5,2
forebrain_P0_1,P0,1
forebrain_E14.5_1,E14.5,1
forebrain_E10.5_2,E10.5,2


## Select Pair

In [8]:
time1, time2 = sample_pair

In [9]:
sample_to_use = sample_df[sample_df['devTime'].isin(sample_pair)]
gene_count_to_use = gene_count_df[sample_to_use.index]

sample_to_use

,devTime,rep
forebrain_E10.5_1,E10.5,1
forebrain_E14.5_2,E14.5,2
forebrain_E14.5_1,E14.5,1
forebrain_E10.5_2,E10.5,2


In [10]:
gene_count_to_use.head()

,forebrain_E10.5_1,forebrain_E14.5_2,forebrain_E14.5_1,forebrain_E10.5_2
gene_id,,,,
ENSMUSG00000000001.4,16156,9459,10751,14368
ENSMUSG00000000028.15,3997,1038,1428,3666
ENSMUSG00000000031.16,4061,909,1059,4205
ENSMUSG00000000037.17,355,225,289,283
ENSMUSG00000000049.11,0,4,0,0


## Run DESeq2, this is very simple R code!

the DESeq2 code is very simple, but it does a lot of work behind! See the above suggested reading for more details

In [19]:
%%R -i gene_count_to_use -i sample_to_use -i time1 -i time2 -o deg -o counts
# this "%%R" line must be the first line of this cell, because it tells jupyter this is a R cell
# -i means import python variable into R
# python and R environment is separate! E.g., dds object will not be avaliable outside this cell, because its in R
# search rpy2 ipython magic if you want to learn more


dds <- DESeqDataSetFromMatrix(countData = gene_count_to_use,
                              colData = sample_to_use,
                              design = ~ rep + devTime)
dds <- DESeq(dds)

resultsNames(dds)

res <- results(dds, contrast=c('devTime', time1, time2), filterFun=ihw)

# get the DEG results as a dataframe
deg <- as.data.frame(res)
# get the normalized counts as a dataframe
counts <- as.data.frame(counts(dds, normalized=TRUE))

R[write to console]: estimating size factors

R[write to console]: estimating dispersions

R[write to console]: gene-wise dispersion estimates

R[write to console]: mean-dispersion relationship

R[write to console]: final dispersion estimates

R[write to console]: fitting model and testing



In [21]:
deg.head()

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,weight
ENSMUSG00000000001.4,12317.642241,0.297103,0.126877,2.341665,1.919793e-02,4.375757e-02,1.147604
ENSMUSG00000000028.15,2376.622873,1.353595,0.166360,8.136550,4.067019e-16,3.960103e-15,1.147604
ENSMUSG00000000031.16,2383.991337,1.775855,0.163946,10.831967,2.428815e-27,4.404670e-26,1.151647
ENSMUSG00000000037.17,281.576119,0.015541,0.324316,0.047918,9.617817e-01,9.854592e-01,1.302372
ENSMUSG00000000049.11,1.165938,-2.733085,4.912529,-0.556350,5.779717e-01,1.000000e+00,0.000000


In [22]:
counts.head()

,forebrain_E10.5_1,forebrain_E14.5_2,forebrain_E14.5_1,forebrain_E10.5_2
ENSMUSG00000000001.4,13676.188563,11028.608425,11078.729961,13487.042016
ENSMUSG00000000028.15,3383.493791,1210.243741,1471.530684,3441.223276
ENSMUSG00000000031.16,3437.670324,1059.837727,1091.282209,3947.175089
ENSMUSG00000000037.17,300.510457,262.336071,297.809781,265.648169
ENSMUSG00000000049.11,0.000000,4.663752,0.000000,0.000000


In [25]:
# concatenate these two tables and save it into a file
deg_with_counts = pd.concat([deg, counts], axis=1)
deg_with_counts.to_csv(f'{time1}_vs_{time2}.deg_results.csv.gz')

In [26]:
deg_with_counts

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,weight,forebrain_E10.5_1,forebrain_E14.5_2,forebrain_E14.5_1,forebrain_E10.5_2
ENSMUSG00000000001.4,12317.642241,0.297103,0.126877,2.341665,1.919793e-02,4.375757e-02,1.147604,13676.188563,11028.608425,11078.729961,13487.042016
ENSMUSG00000000028.15,2376.622873,1.353595,0.166360,8.136550,4.067019e-16,3.960103e-15,1.147604,3383.493791,1210.243741,1471.530684,3441.223276
ENSMUSG00000000031.16,2383.991337,1.775855,0.163946,10.831967,2.428815e-27,4.404670e-26,1.151647,3437.670324,1059.837727,1091.282209,3947.175089
ENSMUSG00000000037.17,281.576119,0.015541,0.324316,0.047918,9.617817e-01,9.854592e-01,1.302372,300.510457,262.336071,297.809781,265.648169
ENSMUSG00000000049.11,1.165938,-2.733085,4.912529,-0.556350,5.779717e-01,1.000000e+00,0.000000,0.000000,4.663752,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
ENSMUSG00000118613.1,1.647316,-4.325815,4.307961,-1.004144,3.153090e-01,1.000000e+00,0.000000,0.000000,3.497814,3.091451,0.000000
ENSMUSG00000118619.1,2.460151,0.209040,2.970134,0.070381,9.438906e-01,1.000000e+00,0.000000,1.693017,2.331876,2.060967,3.754744
ENSMUSG00000118626.1,7.283945,3.532301,2.148490,1.644085,1.001586e-01,1.646480e-01,1.300747,5.925558,3.497814,0.000000,19.712408
ENSMUSG00000118633.1,0.469248,-0.297609,4.887647,-0.060890,9.514467e-01,1.000000e+00,0.000000,0.846508,0.000000,1.030484,0.000000
